# Libraries and requirements

In [1]:
!pip install opencv-python-headless
!pip install numpy pandas scikit-learn scikit-image tensorflow
!pip install unrar

In [26]:
import os
from google.colab import drive
import cv2
import numpy as np
import pandas as pd
from collections import defaultdict
from skimage.feature import local_binary_pattern
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from scipy.fft import fft2 
from scipy.signal import find_peaks  
from sklearn.model_selection import train_test_split
from skimage.feature import local_binary_pattern, graycomatrix, graycoprops 
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score

# Training Data

In [5]:
drive.mount('/content/drive')

# Path to the RAR file in Google Drive
rar_path = '/content/drive/MyDrive/CV/CASIA_faceAntisp.rar'

# Create data directory
if not os.path.exists('data'):
    os.makedirs('data')

# Extract RAR file
!unrar x $rar_path data/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/drive/MyDrive/CV/CASIA_faceAntisp.rar

You can download the datset from here : https://pan.baidu.com/s/15HyX7tizCCuwN9BKiV9_zA
Password: h5un
however, the labels are missing!

see https://github.com/mnikitin/Learn-Convolutional-Neural-Network-for-Face-Anti-Spoofing/issues/5#issuecomment-667916800

Creating    data/test_release                                         OK
Creating    data/test_release/18                                      OK
Extracting  data/test_release/18/1.avi                                     0%  OK 
Creating    data/train_release                                        OK
Creating    data/train_release/19                                     OK
Extracting  data/train_release/19/1.avi                                    0% 

# Preprocessing

In [6]:
# Preprocessing functions
def resize_image(image, size=(224, 224)):
    return cv2.resize(image, size)

def histogram_equalization(image):
    # If the image has three channels (color image), equalize each channel separately
    if len(image.shape) == 3:
        for i in range(3):
            image[:,:,i] = cv2.equalizeHist(image[:,:,i])
    else:
        # If the image is grayscale, equalize the histogram directly
        image = cv2.equalizeHist(image)
    return image

def gaussian_blur(image, kernel_size=(5, 5)):
    # The input image to be blurred.
    return cv2.GaussianBlur(image, kernel_size, 0)

def augment_image(image):
    augmented_images = []
    augmented_images.append(cv2.flip(image, 1))  # Horizontal flip
    augmented_images.append(cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE))  # Rotate 90 degrees
    augmented_images.append(cv2.rotate(image, cv2.ROTATE_180))  # Rotate 180 degrees
    augmented_images.append(cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE))  # Rotate 270 degrees
    return augmented_images

# Feature Extraction Model

In [7]:
# Feature Extraction functions
def extract_frequency_features(image, target_size=(224, 224)):
    resized_image = cv2.resize(image, target_size)  # Resize image to target size
    gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)  # Convert image to grayscale
    f_transform = np.fft.fft2(gray_image)  # Compute 2D FFT of the grayscale image
    f_shifted = np.fft.fftshift(f_transform)  # Shift zero frequency component to the center
    magnitude_spectrum = 20 * np.log(np.abs(f_shifted) + 1e-10)  # Compute magnitude spectrum
    return magnitude_spectrum.flatten()  # Flatten the magnitude spectrum to 1D array


class FeatureModel:
    def __init__(self):
        self.model = SVC(kernel='linear', probability=True)

    def train(self, X, y):
        # Train the SVM classifier
        self.model.fit(X, y)

    def predict(self, features):
        return self.model.predict_proba([features])[0, 1]

    def extract_features(self, data):
        if data is not None:
            image = data.get('image')  # Retrieve image from data dictionary
        feature_list = []  # Initialize list to store features
        if image is not None:
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert image to grayscale
            feature_list.extend(self.frequency_features(gray_image))  # Extract frequency features
            feature_list.extend(self.texture_features(gray_image))  # Extract texture features
            feature_list.extend(self.glcm_features(gray_image))  # Extract GLCM features
        return np.array(feature_list)  # Convert list of features to NumPy array

    def frequency_features(self, gray_image):
        f_transform = fft2(gray_image)  # Compute 2D FFT of the grayscale image
        f_shifted = np.fft.fftshift(f_transform)  # Shift zero frequency component to the center
        magnitude_spectrum = 20 * np.log(np.abs(f_shifted) + 1e-6)  # Compute magnitude spectrum
        return magnitude_spectrum.flatten()  # Flatten the magnitude spectrum to 1D array


    def texture_features(self, gray_image):
        lbp_image = local_binary_pattern(gray_image, P=8, R=1, method="uniform")  # Compute LBP
        (hist, _) = np.histogram(lbp_image.ravel(), bins=np.arange(0, 59))  # Compute histogram of LBP
        hist = hist.astype("float")  # Convert histogram to float type
        hist /= (hist.sum() + 1e-6)  # Normalize the histogram
        return hist  # Return the histogram as texture features

    def glcm_features(self, gray_image):
        glcm = graycomatrix(gray_image, distances=[5], angles=[0], levels=256, symmetric=True, normed=True)  # Compute GLCM
        contrast = graycoprops(glcm, 'contrast')[0, 0]  # Extract contrast feature
        dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]  # Extract dissimilarity feature
        homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]  # Extract homogeneity feature
        energy = graycoprops(glcm, 'energy')[0, 0]  # Extract energy feature
        correlation = graycoprops(glcm, 'correlation')[0, 0]  # Extract correlation feature
        return np.array([contrast, dissimilarity, homogeneity, energy, correlation])  # Return array of GLCM features

# Transfer Learning Model

In [8]:
class TransferLearningModel:
    def __init__(self, input_shape):
        # Load the VGG16 model with pre-trained ImageNet weights, excluding the top fully connected layers
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(1024, activation='relu')(x)
        x = Dropout(0.5)(x)
        predictions = Dense(1, activation='sigmoid')(x)

        # Define the complete model
        self.model = Model(inputs=base_model.input, outputs=predictions)

        # Freeze the layers of the base model to prevent them from being trained
        for layer in base_model.layers:
            layer.trainable = False

        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    def train(self, X, y, epochs=10, batch_size=32):
        self.model.fit(X, y, epochs=epochs, batch_size=batch_size)

    def predict(self, image):
        # Expand dimensions of the image to match the input shape expected by the model
        return self.model.predict(np.expand_dims(image, axis=0))[0, 0]

# Getting frames from videoes and labeling them

In [9]:
def load_videos_with_labels(data_dir):
    data = []  # List to store preprocessed video frames
    labels = []  # List to store labels for each frame
    label_mapping = {
        '1.avi': 1, '2.avi': 1, 'HR_1.avi': 1, 'HR_4.avi': 1,
        '3.avi': 0, '4.avi': 0, '5.avi': 0, '6.avi': 0,
        '7.avi': 0, '8.avi': 0, 'HR_2.avi': 0, 'HR_3.avi': 0
    }

    # Iterate over all subdirectories in the data directory
    for subdir in os.listdir(data_dir):
        subdir_path = os.path.join(data_dir, subdir)  # Get the full path of the subdirectory
        if os.path.isdir(subdir_path):  # Check if the path is a directory
            # Iterate over all files in the subdirectory
            for filename in os.listdir(subdir_path):
                if filename in label_mapping and filename.endswith('.avi'):  # Check if the file is in the label mapping and is an .avi file
                    video_path = os.path.join(subdir_path, filename)
                    cap = cv2.VideoCapture(video_path)  # Open the video file using OpenCV
                    fps = cap.get(cv2.CAP_PROP_FPS)  # Get frames per second of the video
                    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Get total number of frames in the video
                    duration = frame_count / fps  # Calculate duration of the video in seconds

                    # Extract frames from the video at 12-second intervals
                    for sec in range(0, int(duration), 12):
                        cap.set(cv2.CAP_PROP_POS_MSEC, sec * 1000)  # Set the position of the video to the specified time in milliseconds
                        ret, frame = cap.read()  # Read the frame at the specified position
                        if ret:  # Check if the frame was successfully read
                            # Apply preprocessing steps to the frame
                            frame = resize_image(frame, (224, 224))
                            frame = histogram_equalization(frame)
                            frame = gaussian_blur(frame)

                            data.append({'image': frame})  # Append the preprocessed frame to the data list
                            labels.append(label_mapping[filename])  # Append the corresponding label to the labels list

                            # Perform data augmentation on the frame
                            augmented_frames = augment_image(frame)  # Augment the frame
                            for aug_frame in augmented_frames:
                                data.append({'image': aug_frame})  # Append the augmented frame to the data list
                                labels.append(label_mapping[filename])  # Append the corresponding label to the labels list

                    cap.release()
    return data, np.array(labels)

# Training

**feature-based model**

In [10]:
# Load data for feature-based model
x_train, y_train = load_videos_with_labels('data/train_release')
x_test, y_test = load_videos_with_labels('data/test_release')

# Extract features using defined FeatureModel
feature_model = FeatureModel()
x_features = np.array([feature_model.extract_features(x) for x in x_train])

# Train the feature model
feature_model.train(x_features, y_train)

**VGG16 model**

In [11]:
# Convert to numpy array and preprocess data for VGG16
x_vgg = np.array([data['image'] for data in x_train], dtype=np.float32)
x_vgg = preprocess_input(x_vgg)

# Initialize and train the transfer learning model
input_shape = (224, 224, 3)
transfer_model = TransferLearningModel(input_shape)
transfer_model.train(x_vgg, y_train, epochs=10, batch_size=32)

58889256/58889256 [==============================] - 2s 0us/step
Epoch 1/10
40/40 [==============================] - 17s 167ms/step - loss: 0.6270 - accuracy: 0.7992
Epoch 2/10
40/40 [==============================] - 5s 113ms/step - loss: 0.2661 - accuracy: 0.8853
Epoch 3/10
40/40 [==============================] - 5s 115ms/step - loss: 0.1855 - accuracy: 0.9052
Epoch 4/10
40/40 [==============================] - 5s 114ms/step - loss: 0.1786 - accuracy: 0.9068
Epoch 5/10
40/40 [==============================] - 5s 115ms/step - loss: 0.1552 - accuracy: 0.9267
Epoch 6/10
40/40 [==============================] - 5s 117ms/step - loss: 0.1658 - accuracy: 0.9163
Epoch 7/10
40/40 [==============================] - 5s 116ms/step - loss: 0.1504 - accuracy: 0.9235
Epoch 8/10
40/40 [==============================] - 5s 115ms/step - loss: 0.1502 - accuracy: 0.9171
Epoch 9/10
40/40 [==============================] - 5s 118ms/step - loss: 0.1507 - accuracy: 0.9211
Epoch 10/10
40/40 [===============

# Loading Test Data

In [13]:
def resize_image(image, target_size):
    return cv2.resize(image, target_size, interpolation=cv2.INTER_AREA)

def load_test_data_one_frame(data_dir):
    test_data = []  # List to store test data
    video_files = []  # List to store video file names
    labels = []  # List to store labels for each frame

    for filename in os.listdir(data_dir):
        if filename.endswith('.mp4'):  # Check if the file is an .mp4 video file
            video_path = os.path.join(data_dir, filename)  # Get the full path of the video file
            cap = cv2.VideoCapture(video_path)  # Open the video file using OpenCV
            label = 1 if 'live' in filename else 0

            ret, frame = cap.read()  # Read the first frame of the video
            if ret:  # Check if the frame was successfully read
                frame = resize_image(frame, (224, 224)) # preprocessing
                frame = histogram_equalization(frame)
                frame = gaussian_blur(frame)
                test_data.append({'image': frame})
                video_files.append(filename)  # Append the video file name to the video files list
                labels.append(label)  # Append the corresponding label to the labels list

                # augmentation
                augmented_imgs = augment_image(frame)
                for ag in augmented_imgs:
                    ag = resize_image(ag, (224, 224))
                    # preprocessing
                    ag = histogram_equalization(ag)
                    ag = gaussian_blur(ag)
                    test_data.append({'image': ag})
                    video_files.append(filename)  # Append the video file name to the video files list
                    labels.append(label)
            cap.release()

    return test_data, video_files, np.array(labels)


data_dir = '/content/drive/MyDrive/DATASET'
frames, files, labels = load_test_data_one_frame(data_dir)

# Cropping Face Function

In [14]:
def crop_face(frame):
    # Ensure the frame is in the correct format (numpy array)
    resized_frame = np.array(frame, dtype=np.uint8)

    # Check if all pixels are black
    if not np.any(resized_frame):
        # If the frame is completely black, return None as no face can be detected
        return None

    # Convert the frame to grayscale
    grayscale_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)

    # Load the pre-trained Haar Cascade face detector
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Detect faces in the grayscale frame
    faces = face_cascade.detectMultiScale(grayscale_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # If no faces are found, return the resized frame
    if len(faces) == 0:
        return resized_frame

    # Get the coordinates of the first face detected
    (x, y, width, height) = faces[0]

    # Crop the face from the frame using the coordinates
    cropped_face_frame = resized_frame[y:y+height, x:x+width]

    return cropped_face_frame

# List to store the cropped faces
cropped_faces_list = []

# Iterate over the training frames
for index, frame_data in enumerate(x_train):
    if isinstance(frame_data, dict) and 'image' in frame_data:
        frame = frame_data['image']  # Extract the image if it's wrapped in a dictionary
    else:
        frame = frame_data

    # Crop the face from the current frame
    cropped_face = crop_face(frame)

    # If a face was successfully cropped, append it to the list of cropped faces
    if cropped_face is not None:
        cropped_faces_list.append(cropped_face)

# Evaluation on test_Release data giving the full image

**feature-based model**

In [15]:
# Evaluate the feature-based model
x_test_features = np.array([feature_model.extract_features(x) for x in x_test])

# Predict probabilities using the trained feature model
y_pred_probabilities = [feature_model.predict(x) for x in x_test_features]

# Convert probabilities to binary predictions (1 if probability >= 0.5, else 0)
y_pred_binary = [1 if probability >= 0.5 else 0 for probability in y_pred_probabilities]

# Calculate the accuracy of the feature-based model
feature_model_accuracy = accuracy_score(y_test, y_pred_binary)

# Print the accuracy of the feature-based model
print(f"Feature Model's Accuracy with the whole image is: {feature_model_accuracy}")

Feature Model's Accuracy with the whole image is: 0.7403314917127072


**VGG16 model**

In [16]:
# Evaluate the deep learning model
x_test_deep_learning = np.array([data['image'] for data in x_test], dtype=np.float32)

# Preprocess the image data for input to the VGG16 model (e.g., scaling pixel values)
x_test_deep_learning = preprocess_input(x_test_deep_learning)

# Predict probabilities using the trained transfer learning model
y_pred_deep_learning = [transfer_model.model.predict(np.expand_dims(image, axis=0), verbose=0)[0, 0] for image in x_test_deep_learning]

# Convert probabilities to binary predictions (1 if probability >= 0.5, else 0)
y_pred_deep_learning_binary = [1 if probability >= 0.5 else 0 for probability in y_pred_deep_learning]

# Calculate the accuracy of the transfer learning model
deep_learning_model_accuracy = accuracy_score(y_test, y_pred_deep_learning_binary)

# Print the accuracy of the transfer learning model
print(f"Transfer Learning Model's Accuracy with the whole image is: {deep_learning_model_accuracy}")

Transfer Learning Model's Accuracy with the whole image is: 0.9060773480662984


# Evaluation of two models on Full Image input

**feature-based model**

In [17]:
# Evaluate the feature-based model
X_features_test = np.array([feature_model.extract_features(x) for x in frames])

# Predict probabilities using the trained feature model
y_pred = [feature_model.predict(x) for x in X_features_test]

y_pred_binary = [1 if p >= 0.5 else 0 for p in y_pred]

# Calculate the accuracy of the feature-based model
feature_model_accuracy = accuracy_score(labels, y_pred_binary)

print(f"Feature Model's Accuracy with the whole image is {feature_model_accuracy}")

Feature Model's Accuracy with the whole image is 0.6


**VGG16 model**

In [18]:
# Evaluate the deep learning model
X_test_dl = np.array([data['image'] for data in frames], dtype=np.float32)

X_test_dl = preprocess_input(X_test_dl)

# Predict probabilities using the trained transfer learning model
y_pred_dl = [transfer_model.model.predict(np.expand_dims(x, axis=0), verbose=0)[0, 0] for x in X_test_dl]

y_pred_dl_binary = [1 if p >= 0.5 else 0 for p in y_pred_dl]

# Calculate the accuracy of the transfer learning model
dl_accuracy = accuracy_score(labels, y_pred_dl_binary)

print(f"Transfer Learning Model's Accuracy with the whole image is: {dl_accuracy}")

Transfer Learning Model's Accuracy with the whole image is: 0.76


# Results of the 3 cases

**Pad features to align size**

In [19]:
# Function to pad or truncate feature vectors to a specified target length for frequency uses
def pad_or_truncate_features(feature_vector, target_length):
    current_length = len(feature_vector)

    if current_length < target_length:
        # Pad with zeros if the current length is less than the target length
        return np.pad(feature_vector, (0, target_length - current_length), 'constant')
    elif current_length > target_length:
        # Truncate if the current length is greater than the target length
        return feature_vector[:target_length]
    else:
        # Return the original feature vector if it is already the target length
        return feature_vector

**Store the results**

In [27]:
feature_results = defaultdict(list)
deep_results = defaultdict(list)

# Main loop for processing frames
for i, file in enumerate(files):
    frame = frames[i]

    full_image_result = feature_model.predict(feature_model.extract_features(frame))
    face_cropped = crop_face(frame['image'])
    if face_cropped is not None:
        face_cropped_resized = resize_image(face_cropped, (224, 224))
        dic = {'image': face_cropped_resized}
        cropped_image_result = feature_model.predict(feature_model.extract_features(dic))
        deep_cropped_image_result = transfer_model.model.predict(np.expand_dims(face_cropped_resized, axis=0), verbose=0)[0, 0]
    else:
        cropped_image_result = None
        deep_cropped_image_result = None

    deep_full_image_result = transfer_model.model.predict(np.expand_dims(frame['image'], axis=0), verbose=0)[0, 0]

    # Frequency features evaluation
    features = extract_frequency_features(frame['image'])
    features_padded = pad_or_truncate_features(features, 50239)
    freq_result = feature_model.predict(features_padded)
    deep_freq_result = feature_model.predict(features_padded)

    # Save results in dictionaries
    feature_results[file].append([full_image_result, cropped_image_result, freq_result])
    deep_results[file].append([deep_full_image_result, deep_cropped_image_result, deep_freq_result])

# Calculate the mean of the results for each frame name
mean_feature_results = {file: np.mean(results, axis=0) for file, results in feature_results.items()}
mean_deep_results = {file: np.mean(results, axis=0) for file, results in deep_results.items()}

# Convert the results to lists
feature_results_list = [[file] + list(results) for file, results in mean_feature_results.items()]
deep_results_list = [[file] + list(results) for file, results in mean_deep_results.items()]

# CSV

In [28]:
# Save the results to CSV files
pd.DataFrame(feature_results_list, columns=['Video', 'Full Image', 'Cropped Face', 'Frequency']).to_csv('processed_feature_predictions.csv', index=False)
pd.DataFrame(deep_results_list, columns=['Video', 'Deep Full Image', 'Deep Cropped Face', 'Deep Frequency']).to_csv('processed_deep_predictions.csv', index=False)